# TensorFlow - Pipeline Review

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
  
nltk.download('stopwords')

import string

## Get data

In [ ]:
import pandas as pd
yelp_ratings = pd.read_csv("yelp_ratings.csv") #44530 samples
yelp_reviews = yelp_ratings["text"].values
yelp_reviews_sentiment = yelp_ratings["sentiment"].values
yelp_reviews_stars = yelp_ratings["stars"].values

In [ ]:
yelp_reviews[0].lower()

## Custom preprocess data

In [ ]:
# Convert text to lowercase, tokenize, remove stopwords/punctuation, 

stpwrds = stopwords.words('english') + list(string.punctuation)

processed_text = []
for review in yelp_reviews:
    processed_text.append([word for word in word_tokenize(review.lower()) 
                           if word not in stpwrds])

## Train/Test split

In [ ]:
training_sentences = processed_text[0:20000]
testing_sentences = processed_text[20000:]

training_sentiment = yelp_reviews_sentiment[0:20000]
testing_sentiment = yelp_reviews_sentiment[20000:]

## Fit the text to prepare to convert it to number sequences

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<oov>")
tokenizer.fit_on_texts(training_sentences)

## Convert text to number sequences and add padding

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=150, 
                                padding="post", truncating="post")

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=150, 
                                padding="post", truncating="post")

In [ ]:
training_padded.shape

## Create a Model (Neural Network)

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Embedding(10000, 16, input_length=150),
    tf.keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model

In [ ]:
history = model.fit(training_padded, training_sentiment, validation_split=.1, epochs=5) 

## Evaluate the model on the test set

In [ ]:
test_accuracy = model.evaluate(testing_padded, testing_sentiment, verbose=2)[1]

In [ ]:
print("The accuracy on the test set is: {0:.2f}".format(test_accuracy))

## Learning curve

In [ ]:
# plot learning curves
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.title('Training and validation')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

plot_graphs(history, "accuracy")
plot_graphs(history, "loss")